<a href="https://colab.research.google.com/github/leonnmarcoo/CCMACLRL_EXERCISES_COM232/blob/main/Exercise6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 6: Choosing the best performing model on a dataset

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
- Use all Regression models

Submit your results to:
https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview



In [419]:
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import root_mean_squared_error

## Dataset File

In [420]:
train_data = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/3fd7d51ffd17863598ac3f44eeefc558171a5b73/dataset/house-prices-advanced-regression-techniques/train.csv?raw=true'
df = pd.read_csv(train_data)

In [421]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [422]:
pd.set_option('display.max_columns', None)
df.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,GasA,Ex,

## Data Cleaning

In [423]:
label_encoder = LabelEncoder()
df['MSZoning'] = label_encoder.fit_transform(df['MSZoning'])
df['Street'] = label_encoder.fit_transform(df['Street'])
df['LotShape'] = label_encoder.fit_transform(df['LotShape'])
df['LandContour'] = label_encoder.fit_transform(df['LandContour'])
df['Utilities'] = label_encoder.fit_transform(df['Utilities'])
df['LotConfig'] = label_encoder.fit_transform(df['LotConfig'])
df['LandSlope'] = label_encoder.fit_transform(df['LandSlope'])
df['Neighborhood'] = label_encoder.fit_transform(df['Neighborhood'])
df['Neighborhood'] = label_encoder.fit_transform(df['Neighborhood'])
df['Condition1'] = label_encoder.fit_transform(df['Condition1'])
df['Condition2'] = label_encoder.fit_transform(df['Condition2'])
df['BldgType'] = label_encoder.fit_transform(df['BldgType'])
df['HouseStyle'] = label_encoder.fit_transform(df['HouseStyle'])
df['RoofStyle'] = label_encoder.fit_transform(df['RoofStyle'])
df['RoofMatl'] = label_encoder.fit_transform(df['RoofMatl'])
df['Exterior1st'] = label_encoder.fit_transform(df['Exterior1st'])
df['Exterior2nd'] = label_encoder.fit_transform(df['Exterior2nd'])
df['MasVnrType'] = label_encoder.fit_transform(df['MasVnrType'])
df['ExterQual'] = label_encoder.fit_transform(df['ExterQual'])
df['ExterCond'] = label_encoder.fit_transform(df['ExterCond'])
df['Foundation'] = label_encoder.fit_transform(df['Foundation'])
df['BsmtQual'] = label_encoder.fit_transform(df['BsmtQual'])
df['BsmtCond'] = label_encoder.fit_transform(df['BsmtCond'])
df['BsmtExposure'] = label_encoder.fit_transform(df['BsmtExposure'])
df['BsmtFinType1'] = label_encoder.fit_transform(df['BsmtFinType1'])
df['BsmtFinType2'] = label_encoder.fit_transform(df['BsmtFinType2'])
df['Heating'] = label_encoder.fit_transform(df['Heating'])
df['HeatingQC'] = label_encoder.fit_transform(df['HeatingQC'])
df['CentralAir'] = label_encoder.fit_transform(df['CentralAir'])
df['Electrical'] = label_encoder.fit_transform(df['Electrical'])
df['KitchenQual'] = label_encoder.fit_transform(df['KitchenQual'])
df['Functional'] = label_encoder.fit_transform(df['Functional'])
df['FireplaceQu'] = label_encoder.fit_transform(df['FireplaceQu'])
df['GarageType'] = label_encoder.fit_transform(df['GarageType'])
df['GarageFinish'] = label_encoder.fit_transform(df['GarageFinish'])
df['GarageQual'] = label_encoder.fit_transform(df['GarageQual'])
df['GarageCond'] = label_encoder.fit_transform(df['GarageCond'])
df['PavedDrive'] = label_encoder.fit_transform(df['PavedDrive'])
df['PoolQC'] = label_encoder.fit_transform(df['PoolQC'])
df['Fence'] = label_encoder.fit_transform(df['Fence'])
df['MiscFeature'] = label_encoder.fit_transform(df['MiscFeature'])
df['SaleType'] = label_encoder.fit_transform(df['SaleType'])
df['SaleCondition'] = label_encoder.fit_transform(df['SaleCondition'])


In [424]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int64  
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int64  
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   int64  
 8   LandContour    1460 non-null   int64  
 9   Utilities      1460 non-null   int64  
 10  LotConfig      1460 non-null   int64  
 11  LandSlope      1460 non-null   int64  
 12  Neighborhood   1460 non-null   int64  
 13  Condition1     1460 non-null   int64  
 14  Condition2     1460 non-null   int64  
 15  BldgType       1460 non-null   int64  
 16  HouseStyle     1460 non-null   int64  
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [425]:
pd.set_option('display.max_rows', None)
df.isnull().sum()

,0
Id,0
MSSubClass,0
MSZoning,0
LotFrontage,259
LotArea,0
Street,0
Alley,1369
LotShape,0
LandContour,0
Utilities,0


In [426]:
median_imputer = SimpleImputer(strategy='median')
df['LotFrontage'] = median_imputer.fit_transform(df[['LotFrontage']])
df['MasVnrArea'] = median_imputer.fit_transform(df[['MasVnrArea']])
df['GarageYrBlt'] = median_imputer.fit_transform(df[['GarageYrBlt']])

drop_col=['Alley']
df.drop(drop_col, axis=1, inplace=True)

In [427]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int64  
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int64  
 6   LotShape       1460 non-null   int64  
 7   LandContour    1460 non-null   int64  
 8   Utilities      1460 non-null   int64  
 9   LotConfig      1460 non-null   int64  
 10  LandSlope      1460 non-null   int64  
 11  Neighborhood   1460 non-null   int64  
 12  Condition1     1460 non-null   int64  
 13  Condition2     1460 non-null   int64  
 14  BldgType       1460 non-null   int64  
 15  HouseStyle     1460 non-null   int64  
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

## Test File

In [428]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/3fd7d51ffd17863598ac3f44eeefc558171a5b73/dataset/house-prices-advanced-regression-techniques/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [429]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

### Data Cleaning

In [430]:
label_encoder = LabelEncoder()
dt['MSZoning'] = label_encoder.fit_transform(dt['MSZoning'])
dt['Street'] = label_encoder.fit_transform(dt['Street'])
dt['LotShape'] = label_encoder.fit_transform(dt['LotShape'])
dt['LandContour'] = label_encoder.fit_transform(dt['LandContour'])
dt['Utilities'] = label_encoder.fit_transform(dt['Utilities'])
dt['LotConfig'] = label_encoder.fit_transform(dt['LotConfig'])
dt['LandSlope'] = label_encoder.fit_transform(dt['LandSlope'])
dt['Neighborhood'] = label_encoder.fit_transform(dt['Neighborhood'])
dt['Neighborhood'] = label_encoder.fit_transform(dt['Neighborhood'])
dt['Condition1'] = label_encoder.fit_transform(dt['Condition1'])
dt['Condition2'] = label_encoder.fit_transform(dt['Condition2'])
dt['BldgType'] = label_encoder.fit_transform(dt['BldgType'])
dt['HouseStyle'] = label_encoder.fit_transform(dt['HouseStyle'])
dt['RoofStyle'] = label_encoder.fit_transform(dt['RoofStyle'])
dt['RoofMatl'] = label_encoder.fit_transform(dt['RoofMatl'])
dt['Exterior1st'] = label_encoder.fit_transform(dt['Exterior1st'])
dt['Exterior2nd'] = label_encoder.fit_transform(dt['Exterior2nd'])
dt['MasVnrType'] = label_encoder.fit_transform(dt['MasVnrType'])
dt['ExterQual'] = label_encoder.fit_transform(dt['ExterQual'])
dt['ExterCond'] = label_encoder.fit_transform(dt['ExterCond'])
dt['Foundation'] = label_encoder.fit_transform(dt['Foundation'])
dt['BsmtQual'] = label_encoder.fit_transform(dt['BsmtQual'])
dt['BsmtCond'] = label_encoder.fit_transform(dt['BsmtCond'])
dt['BsmtExposure'] = label_encoder.fit_transform(dt['BsmtExposure'])
dt['BsmtFinType1'] = label_encoder.fit_transform(dt['BsmtFinType1'])
dt['BsmtFinType2'] = label_encoder.fit_transform(dt['BsmtFinType2'])
dt['Heating'] = label_encoder.fit_transform(dt['Heating'])
dt['HeatingQC'] = label_encoder.fit_transform(dt['HeatingQC'])
dt['CentralAir'] = label_encoder.fit_transform(dt['CentralAir'])
dt['Electrical'] = label_encoder.fit_transform(dt['Electrical'])
dt['KitchenQual'] = label_encoder.fit_transform(dt['KitchenQual'])
dt['Functional'] = label_encoder.fit_transform(dt['Functional'])
dt['FireplaceQu'] = label_encoder.fit_transform(dt['FireplaceQu'])
dt['GarageType'] = label_encoder.fit_transform(dt['GarageType'])
dt['GarageFinish'] = label_encoder.fit_transform(dt['GarageFinish'])
dt['GarageQual'] = label_encoder.fit_transform(dt['GarageQual'])
dt['GarageCond'] = label_encoder.fit_transform(dt['GarageCond'])
dt['PavedDrive'] = label_encoder.fit_transform(dt['PavedDrive'])
dt['PoolQC'] = label_encoder.fit_transform(dt['PoolQC'])
dt['Fence'] = label_encoder.fit_transform(dt['Fence'])
dt['MiscFeature'] = label_encoder.fit_transform(dt['MiscFeature'])
dt['SaleType'] = label_encoder.fit_transform(dt['SaleType'])
dt['SaleCondition'] = label_encoder.fit_transform(dt['SaleCondition'])


In [431]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1459 non-null   int64  
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   int64  
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   int64  
 8   LandContour    1459 non-null   int64  
 9   Utilities      1459 non-null   int64  
 10  LotConfig      1459 non-null   int64  
 11  LandSlope      1459 non-null   int64  
 12  Neighborhood   1459 non-null   int64  
 13  Condition1     1459 non-null   int64  
 14  Condition2     1459 non-null   int64  
 15  BldgType       1459 non-null   int64  
 16  HouseStyle     1459 non-null   int64  
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [432]:
pd.set_option('display.max_rows', None)
dt.isnull().sum()

,0
Id,0
MSSubClass,0
MSZoning,0
LotFrontage,227
LotArea,0
Street,0
Alley,1352
LotShape,0
LandContour,0
Utilities,0


In [433]:
median_imputer = SimpleImputer(strategy='median')
dt['LotFrontage'] = median_imputer.fit_transform(dt[['LotFrontage']])
dt['MasVnrArea'] = median_imputer.fit_transform(dt[['MasVnrArea']])
dt['GarageYrBlt'] = median_imputer.fit_transform(dt[['GarageYrBlt']])

drop_col=['Alley']
dt.drop(drop_col, axis=1, inplace=True)

In [434]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1459 non-null   int64  
 3   LotFrontage    1459 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   int64  
 6   LotShape       1459 non-null   int64  
 7   LandContour    1459 non-null   int64  
 8   Utilities      1459 non-null   int64  
 9   LotConfig      1459 non-null   int64  
 10  LandSlope      1459 non-null   int64  
 11  Neighborhood   1459 non-null   int64  
 12  Condition1     1459 non-null   int64  
 13  Condition2     1459 non-null   int64  
 14  BldgType       1459 non-null   int64  
 15  HouseStyle     1459 non-null   int64  
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearBuil

## Sample Submission File

In [435]:
sample_submission_url ='https://github.com/robitussin/CCMACLRL_EXERCISES/blob/3fd7d51ffd17863598ac3f44eeefc558171a5b73/dataset/house-prices-advanced-regression-techniques/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [436]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id         1459 non-null   int64  
 1   SalePrice  1459 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 22.9 KB


# Model Training

## 1. Train a KNN Regressor

In [437]:
knn = KNeighborsRegressor()
knn.fit(x_train,y_train)
knn_score = knn.score(x_test,y_test)
score_list["KNN"]=knn_score

print(f"Score is {knn_score}")

Score is 0.6064722221534031


In [438]:
y_pred_knn = knn.predict(x_test)

rmse_knn = root_mean_squared_error(y_test, y_pred_knn)

print(f"Root Mean Squared Error for KNN Regressor: {rmse_knn}")

Root Mean Squared Error for KNN Regressor: 53008.560657304086


- Perform cross validation

In [439]:
clf = knn
scores = cross_val_score(clf, x_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
scores

array([-44301.33084624, -44707.37951754, -57842.18586353, -43410.47421739,
       -43036.98228274])

## 2. Train a SVM Regression



In [440]:
svm = SVR()
svm.fit(x_train,y_train)
svm_score = svm.score(x_test,y_test)
score_list["SVM"]=svm_score

print(f"Score is {svm_score}")

Score is -0.03770215337983762


In [441]:
y_pred_svm = svm.predict(x_test)

rmse_svm = root_mean_squared_error(y_test, y_pred_svm)

print(f"Root Mean Squared Error for SVM Regression: {rmse_svm}")

Root Mean Squared Error for SVM Regression: 86078.49682048362


- Perform cross validation

In [442]:
clf = svm
scores = cross_val_score(clf, x_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
scores

array([-73828.6330393 , -76103.03738871, -93367.50825651, -74644.42762872,
       -76049.09700806])

## 3. Train a Decision Tree Regression

In [443]:
dtc = DecisionTreeRegressor(random_state=1)
dtc.fit(x_train,y_train)

dtc_score = dtc.score(x_test,y_test)
score_list["DTC"] = dtc_score
print(f"Score is {dtc_score}")

Score is 0.808961613774016


In [444]:
y_pred_dtc = dtc.predict(x_test)

rmse_dtc = root_mean_squared_error(y_test, y_pred_dtc)

print(f"Root Mean Squared Error for Decision Tree Regressor: {rmse_dtc}")

Root Mean Squared Error for Decision Tree Regressor: 36933.344397829394


- Perform cross validation

In [445]:
clf = dtc
scores = cross_val_score(clf, x_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
scores

array([-53722.32323232, -50284.05044307, -54108.51794198, -34596.10046724,
       -30953.31313509])

## 4. Train a Random Forest Regression

In [446]:
rfc = RandomForestClassifier(n_estimators=50,random_state=1)
rfc.fit(x_train,y_train)
rfc_score = rfc.score(x_test,y_test)
score_list["RFC"]=rfc_score

print(f"Score is {rfc_score}")

Score is 0.0045662100456621


In [447]:
y_pred_rfc = rfc.predict(x_test)

rmse_rfc = root_mean_squared_error(y_test, y_pred_rfc)

print(f"Root Mean Squared Error for Random Forest Regression: {rmse_rfc}")

Root Mean Squared Error for Random Forest Regression: 48852.144434968024


- Perform cross validation

In [448]:
clf = rfc
scores = cross_val_score(clf, x_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
scores

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


array([-45398.40109777, -46035.39352016, -44025.06217942, -50067.48201609,
       -41972.16873517])

## 5. Compare all the performance of all regression models

In [449]:
print("Root Mean Squared Error for each model:")
print(f"KNN Regressor: {rmse_knn}")
print(f"SVM Regression: {rmse_svm}")
print(f"Decision Tree Regressor: {rmse_dtc}")
print(f"Random Forest Regressor: {rmse_rfc}")

Root Mean Squared Error for each model:
KNN Regressor: 53008.560657304086
SVM Regression: 86078.49682048362
Decision Tree Regressor: 36933.344397829394
Random Forest Regressor: 48852.144434968024


# 6. Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [450]:
id = sf.pop('Id')
y_pred = rfc.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'Id': id,
    'SalePrice': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
